In [32]:
import numpy as np
import matplotlib.pyplot as plt

import math

<h1>ReadMe.md</h1>

<h5>Simulate the traffic in a intersection with 2 lanes in each route</h5>

<img src="intersection_img.jpg" width="400" height="400"></img>

<h3>Information about car and road</h3>

- Average car acceleration 7.11 km/hr ^ 2

- Average car lenght " 4.7 m "

- Average lenght + gap between car " 5 m " 

- Average car's speed " 19 km/hr " or " 5.28 m/s "

- Speed limit is 56 km/hr

- Acceleration delay 1.02 second

- Intersection length (before departure line) is " 213 m "


<h3>General Traffic light with filter arrow's order</h3>

1. N & S left turn, forward and right turn

2. E & W left turn, forward and right turn

3. repeat

- <a href="https://docs.google.com/spreadsheets/d/1U2yDHrIdD2DLxR9tJUxkQAIjotMM6K5ttKjFmD_xJpg/edit?usp=sharing">click here</a> for the information about time loop

<h3>Program note</h3>

there are 3 possible ways of traffic sign control

1. every lanes in a route

2. 2 right handed lanes with opposite route turn right and the 2 left handed lanes of the rest route allow left

3. 2 left handed lanes with opposite route move forward and turn left


In [33]:
class intersection:
    
    lanes = None
    
    def __init__(self):
        self.lanes = [[] for i in range(8)]
    
    def push(self, index, position, speed):
        self.lanes[index].append([position, speed])
        
    def pop(self, index):
        return self.lanes[index].pop(0)
        
    def empty(self, index):
        if len(self.lanes[index]) != 0:
            return False
        else:
            return True
    
    def clear(self, index):
        self.lanes[index] = []
        
    def clear_all(self):
        self.lanes = [[] for i in range(8)]
    
    def printlane(self, index):
        print(self.lanes[index])
        
    def density(self, index):
        return len(self.lanes[index])
    
    def get_position(self, index):
        return self.lanes[index][0][0]
    
    def get_speed(self, index):
        return self.lanes[index][0][1]
        
    
    
    

In [34]:
# Add 8 lanes into each route of intersection
#
#                                S  E  N  W
# even index is left-handed lane 0, 2, 4, 6
# odd index is right-handed lane 1, 3, 5, 7
#
# (0, 4) -> (1, 5) -> (2, 6) -> (3, 7)
#

simu = intersection()

In [35]:
sim_time = 600 # sec (10 min)

# road_len = 213.0 meter

# 5 cars for buffer a road capacity
buff = 33.0 # meter

# 37 cars is a maximum capacity
cap_lenght = 180.0 # meter

endpoint = 12.0 # meter

# Turning distance in angular (in linear 9.75 meter)
angular_end = math.pi/2 # 1/4 of full circle

delay = 1.02 # sec

#acce = 54.86 * 1e-5 * 1e-6 # meter / mili sec^2
acce = 54.86 * 1e-5 # meter/sec

alpha = acce/9.75 # radius / mili sec^2

car_counter = 0

In [36]:
## Setting up simulation for normal trafficlight management in Thailand

# Determine arriving rate of vehicle (number/second) 
# vehicle speed 0.00528 m / mili sec
# 5 m lenght need 947 mili sec
# E : Thru 0.22, Right 0.06 veh/s
# W : Thru 0.26, Right 0.03 veh/s
# N : Thru 0.08, Right 0.02 veh/s
# S : Thru 0.09, Right 0.04 veh/s

# Determine initial existing vheicles in each lane (+10)
# E : Thru 24, Right 16
# W : Thru 26, Right 13
# N : Thru 17. Right 12
# S : Thru 18, Right 14

def generate():  
    existing = [18, 14, 24, 16, 17, 12, 26, 13]
    
    for i in range(len(existing)):
        dist = 0.0
        for j in range(existing[i]):
            simu.push(i, position=dist, speed=0.0)
            dist = dist + 5.0




In [37]:
# S E N W respectively
rate = [
    (0.09, 0.04), (0.22, 0.06), 
        (0.08, 0.02), (0.26, 0.03)
]
generate()

In [38]:
for i in range(8):
    print(simu.density(i))

18
14
24
16
17
12
26
13


In [39]:
def find_s(u, t, a):
    return u*t + 0.5*a*t^2

def find_v(u, t, a):
    return u + a*t

In [40]:
# Return " True " when the car is not possible to pass the intersection on time limit
def Is_deacc(curr_dist, speed, remain_t, Is_thru):
    if Is_thru:
        if endpoint + curr_dist > find_s(speed, remain_t, acce):
            return True
        return False
    else:
        if curr_dist > find_s(speed, remain_t, acce) and angular_end > find_s(speed/9.75, remain_t, alpha):
            return True
        return False

In [41]:
def general_apporch_simulation():
    
    x_axis = [] # Time in milisecond
    y_axis = [] # Number of passed vehicle
    
    green = [float(14), float(24), float(14), float(22)]
    yell = 2
    
    order = [(0, 4), (1, 5), (2, 6), (3, 7)]
    
    sum_time = 0
    
    while sum_time < sim_time:
        for i, lane1, lane2 in enumerate(order):
            loop = max(simu.density(lane1),simu.density(lane2))
            t = green[i]
            tpass = 0
            for iter in range(loop):
                if not simu.empty(lane1):
                    s = simu.get_position(lane1)
                    if not Is_deacc(s, 0, t, i%2):
                        simu.pop(lane1)

                if not simu.empty(lane2):
                    s = simu.get_position(lane2)
                    if not Is_deacc(s, 0, t, i%2):
                        simu.pop(lane2)
                t = t - 1.02
                tpass = tpass + 1.02
                
                

            sum_time = green[i] + yell           
            
    

In [ ]:
#calculate flow rate 
def flow_rate():
    #flow rate calculation
    flow_rate = []
    for i in range(8):
        flow_rate.append(len(simu.lanes[i])/(sim_time/1000)) #flow rate = number of cars / time
    return flow_rate

#calculate average speed
def avg_speed():
    #average speed calculation
    avg_speed = []
    for i in range(8):
        sum = 0
        for j in range(len(simu.lanes[i])):
            sum += simu.lanes[i][j][1]
        avg_speed.append(sum/len(simu.lanes[i]))
    return avg_speed

print(flow_rate())
print(avg_speed())
